# CHAPTER 12 - Quantifying Uncertainty Example

### Instructor: Brandon Haworth


#### Notebook Credit: George Tzanetakis
Jupyter Notebooks you encounter during the course were largely developed by Prof. Tzanetakis from a previous iteration of this course. I've since changed/developed them where necessary for my own iterations of CSC 421.

# Probabilistic Wumpus World 


<img src="images/prob_wumpus.png" width="75%"/>


* Pits cause breezes in neighbouring squares
* Each square (except [1,1] ) contains a pit with a probability of 0.2
* Random variables P[i,j] for each square B[i,j] for each square that is breezy 
* Please work through examples from the textbook

Suppose that there is a breeze in [1,2] and [2,1]. The possible next moves 
are [1,3],[2,2], and [3,1]. Each of those can potentially have a pit. 
Is there one that is "safer" in a probabilistic sense to move to? 


The full joint distribtion is ${\bf P}(P_{1,1}, \dots, P_{4,4}, B_{1,1}, B_{1,2},B_{2,1})$. Note: we only consider the B variables that are observed. Applying the product rule we get: 
$$ 
{\bf P}(P_{1,1}, \dots, P_{4,4}, B_{1,1}, B_{1,2},B_{2,1}) = {\bf P}((B_{1,1}, B_{1,2},B_{2,1})|(P_{1,1}, \dots, P_{4,4})){\bf P}(P_{1,1}, \dots, P_{4,4})
$$

The first term is the conditional probability distribution of a breeze configuration, given the pit configuration; its values are 1 if the breezes are adjacent to the pits and 0 otherwise. The second term 
is the prior probability of a pit configuration. Each square contains a pit with a probability of $0.2$, independently of the other squares: 

$$
{\bf P}(P_{1,1}, \dots, P_{4,4})=\prod_{i,j=1,1}^{4,4}{\bf P}(P_{i,j})
$$




The probability of a particular configuration with exactly $n$ pits can be 
expressed as follows: 
$$ 
{\bf P}(P_{1,1}, \dots, P_{4,4}) = 0.2^{n} \times 0.8^{16-n}
$$

In the situation in the Figure above, the evidence consists of the observed breeze (or its absence) in each square that is visited, combined with the fact that each such square contains no pit. We abbreviate these facts as $b=\neg b_{1,1} \land b_{1,2} \land b_{2,1}$ and 
$known = \neg p_{1,1} \land \neg p_{1,2} \land \neg p_{2,1}$. 


## Query

How likely is it that [1,3] contains a pit given the observations so far? 
$${\bf P}(P_{1,3}|known,b)$$

We can answer this query by summing over the entries from the full joint distribution. Let $unknown$ be the set of $P_{i,j}$ variables for squares other than the $known$ squares and the query square $[1,3]$. Then we have:

$$ 
{\bf P}(P_{1,3}|known,b) = \alpha\sum_{unknown}{\bf P}(P_{1,3}, unknown, known, b)
$$ 

There are 12 unknown squares therefore this summation contains $2^{12}=4096$ terms. In general, summations like this grow exponentially with the number of squares (or variables more generally). 

From the rules of the Wumpus world, it is clear that many squares are irrelevant to the problem. 
We can formalize this intuition by taking advantage of conditional independence and making solving the problem easier i.e. having fewer terms in the sum. The notation gets a bit complicated but if you take your time you will see it consists of carefully formulating subsets of the variables and using the sum and product rules with conditional independence. 

Let $frontier$ be the pit variables (other than the query variable) that are adjacent to the visited squares. In our case, these will $[2,2]$ and $[3,1]$. Let $other$ be the pit variables for the $10$ other unknown squares. 

**KEY INSIGHT:** The observed breezes are *conditionally independent* of the other variables, given the known, frontier, and query variables.


$$ 
\begin{aligned}
{\bf P}(P_{1,3} | known, b) &= \alpha \sum_{unknown} {\bf P}(P_{1,3}, known, b, unknown) \\ 
&= \alpha \sum_{unknown} P(b | P_{1,3}, known, unknown){\bf P}(P_{1,3}, known, unknown) \\
&= \alpha \sum_{frontier}\sum_{other}{\bf P}(b|P_{1,3}, known, frontier, other){\bf P}(P_{1,3}, known, frontier, other) \\ 
&= \alpha \sum_{frontier}\sum_{other}{\bf P}(b|P_{1,3}, known, frontier){\bf P}(P_{1,3}, known, frontier, other)
\end{aligned} 
$$

The final step uses **conditional independence**: $b$ is conditionally independent of $other$ given $known$, $P_{1,3}$, and $frontier$. 

$$ 
\begin{aligned}
{\bf P}(P_{1,3} | known, b) \\
 &= \alpha \sum_{frontier} {\bf P}(b | P_{1,3}, known, frontier) \sum_{other} {\bf P}(P_{1,3}, known, frontier, other) \\ 
&= \alpha P(known) {\bf P}(P_{1,3}) \sum_{frontier}{\bf P}(b|P_{1,3},known,frontier)P(frontier)\sum_{other}P(other) \\
&= \alpha'{\bf P}(P_{1,3})\sum_{frontier} {\bf P}(b|P_{1,3}, known,frontier)P(frontier)
\end{aligned} 
$$

The last step folds P(known) into the normalizing constant and uses the fact that $\sum_{other}P(other) = 1$.  

There are just 4 terms in the summation over the frontier variables $P_{2,2}$ and $P_{3,1}$. The use of independence and conditional independence has completely eliminated the other squares for consideration. Notice that the expression ${\bf P}(b|P_{1,3}, known, frontier)$ is $1$ when the frontier is consistent with the breeze observations and $0$ otherwise. Thus, for each value of $P_{1,3}$ we sum over the *logical models* for the frontier variables that are consistent with the known facts. 

The models and their associated prior probabilities are shown below. 

<img src="images/consistent_models_prob_wumpus.png" width="75%"/>


We therefore have: 
$$
{\bf P}(P_{1,3}|known,b) = \alpha'<0.2 (0.04 + 0.16 + 0.16), 0.8(0.04 + 0.16))> 
\approx <0.31, 0.69> 
$$

Because of symmetry, we can also infer that $P_{1,3}$ is also $0.31$. A similar 
calculation shows that $[2,2]$ contains a pit with roughly $0.86$ probability.

Therefore the wumpus agent should definitely avoid $[2,2]$. 

In [66]:
p13 = (0.2 * (0.04 + 0.16 + 0.16))
not_p13 = (0.8 * (0.04 + 0.16))

print(p13,not_p13)
print(p13/(p13 + not_p13), not_p13 / (p13 + not_p13))

0.072 0.16000000000000003
0.3103448275862068 0.6896551724137931


<a id='summary'></a> 

# Summary 

* A wumpus-world agent can calculate probabilities for unobserved aspects of the world, thereby improving the decisions of a purely logical agent.
* Conditional independence makes these calculations tractable.